In [1]:
import pandas as pd

# Scraping Wikipedia with Pandas

In [2]:
# Let's first grab the tables
wiki_link = "https://en.wikipedia.org/wiki/List_of_best-selling_game_consoles"
consoles_sold = pd.read_html(wiki_link)
print(f"We found {len(consoles_sold)} tables.")

We found 7 tables.


In [3]:
# Now let's Find the largest one
largest_index = 0

try:
    for i in range(len(consoles_sold)):
        if consoles_sold[i].count().max() > consoles_sold[largest_index].count().max():
            largest_index = i
    print(f"The largest table we grabbed is table {largest_index + 1}.  It has a size of {consoles_sold[largest_index].count().max()}.")
except KeyError:
    print(f"We already found the largest table.  It has a size of {consoles_sold.count().max()} rows.")

The largest table we grabbed is table 1.  It has a size of 40.


In [4]:
# Now to create the pandas DataFrame
try:
    consoles_sold = consoles_sold[largest_index]
    print("Converted the table successfully into a DataFrame.")
except KeyError:
    print("We already converted the table into a DataFrame.")

Converted the table successfully into a DataFrame.


In [5]:
# Save the data to a csv
consoles_sold.to_csv("Wiki_Scrape.csv")

# The DataFrame Table:

In [6]:
consoles_sold.style

,Platform,Firm,Released[2],Units sold,Ref.
0,PlayStation 2,Sony,2000,>155 million,[note 1]
1,Nintendo DS,Nintendo,2004,154.02 million,[17]
2,"Game Boy, Game Boy Color",Nintendo,"1989, 1998",118.69 million,[note 2]
3,PlayStation 4 #,Sony,2013,110.4 million,[32]
4,PlayStation,Sony,1994,102.49 million,[33]
5,Wii,Nintendo,2006,101.63 million,[17]
6,PlayStation 3,Sony,2006,>87.4 million,[note 1]
7,Xbox 360,Microsoft,2005,>84 million,[note 3]
8,Game Boy Advance,Nintendo,2001,81.51 million,[17]
9,PlayStation Portable,Sony,2004,80–82 million(estimate),[note 1]


#### Disclaimer:  We do not own or claim ownership of Wikipedia or any of the information we have sourced from it.  We highly appreciate the work independent contributors have made and will use it responsibly.